In [ ]:
# Implementing the "Cube" approach

In [26]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# added for the Cube approach implementation
from collections import defaultdict
import time
import sgp4
from sgp4.api import Satrec, SatrecArray

In [2]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,c_radius,to_satcat,satcat,debris = pkl.load(file)

In [3]:
cube_dimension = 10 # in km
start = time.time()
# Transform satellite positions r_ic (in SI) to Km
pos = r_ic / 1000
pos = pos / cube_dimension
cubes = defaultdict(list)
pos = np.floor(pos).astype(int)
# We fill the bins
for i, xyz in enumerate(pos):
    cubes[tuple(xyz)].append(i)
# We find te bins with more than one entry
for key in cubes:
    if len(cubes[key]) > 1:
        print("bin is:", cubes[key])
end = time.time()
print("Time elasped: ", end - start)

bin is: [4854, 6471]
bin is: [15565, 15577]
Time elasped:  0.08951830863952637


In [29]:
t0 = 8073.603992389981 #mjd2000

In [4]:
t_curr = t0+5
start = time.time()

for deb in debris:
    try:
        r,v = deb.eph(t_curr)
    except:
        pass
end = time.time()
print("Time elasped: ", end - start)

Time elasped:  0.08301925659179688


In [31]:
# This list will contain all the sgp4 Satrec objects
satellite_l = []
for deb in debris:
    l1 = deb.line1
    l2 = deb.line2
    satellite_l.append(Satrec.twoline2rv(l1, l2))
# Here we build the vectorized version allowing for speed
satellites = SatrecArray(satellite_l)
jd0, fr = pk.epoch(t0).jd, 0.0

In [39]:
# Simulation time (years)
sim_time = 20
# Time steps (days)
time_grid = 5
# The Julian dates are from jd0 to 20 years after
jds = jd0 + np.arange(0,sim_time*365.25/time_grid)*time_grid
frs = jds * 0

In [40]:
start = time.time()
e, r, v = satellites.sgp4(jds, frs)
end = time.time()
print("Time elasped: ", end - start)

Time elasped:  32.45260715484619


In [95]:
np.where(e[:,0]>0)


(array([ 9060, 11626, 13094, 17376, 18679, 20694, 21640, 21705, 21784,
        21839, 21889, 21949, 21985, 22012, 22084, 22166, 22260, 22304,
        22991, 22992, 22993, 22994, 23066, 23122, 23160, 23209]),)

In [115]:
e[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [102]:
e[11626,:]

array([6, 6, 0, ..., 0, 0, 0], dtype=uint8)

array([-1.66942363e+25, -7.07969626e+24, -6.28793769e+25])

In [69]:
r[16,1066,:]

array([ 2640.22338337, -4350.63060018, -3825.20864426])

In [70]:
from sgp4.api import SGP4_ERRORS

In [96]:
SGP4_ERRORS[6]

'mrt is less than 1.0 which indicates the satellite has decayed'